# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846905868835                   -0.70    4.8         
  2   -7.852324919922       -2.27       -1.53    1.0   89.9ms
  3   -7.852616342054       -3.54       -2.56    1.5   30.3ms
  4   -7.852645998424       -4.53       -2.89    2.5   36.3ms
  5   -7.852646507075       -6.29       -3.18    1.2   26.9ms
  6   -7.852646680103       -6.76       -4.13    1.0   24.9ms
  7   -7.852646686692       -8.18       -5.13    2.2   36.3ms
  8   -7.852646686727      -10.46       -5.56    1.5   29.4ms
  9   -7.852646686728      -11.85       -5.63    1.2   29.1ms
 10   -7.852646686730      -11.75       -6.72    1.0   26.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846929698334                   -0.70           4.2         
  2   -7.852555098253       -2.25       -1.64   0.80    2.2    261ms
  3   -7.852638854723       -4.08       -2.75   0.80    1.0   24.6ms
  4   -7.852646572249       -5.11       -3.33   0.80    2.0   90.8ms
  5   -7.852646678700       -6.97       -4.24   0.80    1.5   24.5ms
  6   -7.852646686573       -8.10       -4.82   0.80    2.2   31.2ms
  7   -7.852646686719       -9.83       -5.60   0.80    1.2   28.9ms
  8   -7.852646686730      -10.99       -6.77   0.80    2.2   32.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.417826e+01     3.395009e+00
 * time: 0.06259298324584961
     1     1.631667e+00     1.809978e+00
 * time: 0.29938697814941406
     2    -1.708040e+00     2.153477e+00
 * time: 0.3238658905029297
     3    -3.619673e+00     1.888787e+00
 * time: 0.3589029312133789
     4    -4.966605e+00     1.950443e+00
 * time: 0.39638280868530273
     5    -6.567789e+00     1.142818e+00
 * time: 0.43483495712280273
     6    -7.271658e+00     5.578251e-01
 * time: 0.47048187255859375
     7    -7.549843e+00     3.537144e-01
 * time: 0.49629688262939453
     8    -7.694692e+00     1.159700e-01
 * time: 0.5235958099365234
     9    -7.759558e+00     1.789728e-01
 * time: 0.5486419200897217
    10    -7.801169e+00     8.981334e-02
 * time: 0.575218915939331
    11    -7.823509e+00     7.088640e-02
 * time: 0.5998988151550293
    12    -7.838997e+00     3.711674e-02
 * time: 0.6246368885040283
    13    -7.846775e+00     2.313623e-02
 * time: 0.64965

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846849979637                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645921047                   -1.64         
  2   -7.852646686730       -6.12       -3.71    2.13s
  3   -7.852646686730      -13.34       -7.26    199ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.633080431196351e-7
|ρ_newton - ρ_scfv| = 5.314313581357188e-8
|ρ_newton - ρ_dm|   = 2.509740947988247e-9
